<font size="6">Трансформеры</font>

Порождение выходной последовательности по входной находит применение во многих областях машинного обучения: от обработки естественного языка до генерации описаний объектов на фотографиях.
До недавнего времени наиболее эффективные seq2seq-модели основались на сложных рекуррентных или сверточных нейронных сетях, беря за основу подход encoder-decoder и механизм внимания. В 2017 году в статье [«Attention Is All You Need»](https://arxiv.org/abs/1706.03762), была предложена новая архитектура, основанная исключительно на механизмах внимания, названная
Transformer.

# Классический seq2seq

## Encoder-decoder

В простейшем варианте модель seq2seq представляет собой две последовательно соединенные рекурентные сети: **Encoder** и **Decoder**. Encoder принимает на вход последовательность векторных представлений токенов и генерирует hidden state, который подается на вход Decoder’а. Decoder, в свою очередь, служит для построения целевой последовательности по внутреннему состоянию.

На примере задачи перевода: на вход кодировщику подается текст на исходном языке. Тогда hidden state можно интерпритировать как смысл этого текста, по которому затем декодировщик восстанавливает текст на целевом языке.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn.png" width="800">

При этом возникает следующая **проблема:** мы пропускаем информацию из входной последовательности через бутылочное горлышко — вектор фиксированного размера $h_N$. Что будет, если размер последовательности 1000?

**Идея:** использовать новый вектор контекста на каждом шаге.

**Решение:** — механизм внимания (**Attention**).

# Attention

Концепция **attention** состоит в предположении, что между токенами существуют некоторые взаимосвязи. При таком подходе кодировщик передает в декодировщик не одно состояние, кодирующее всю последовательность целиком, а набор состояний всех токенов. Для этого применяется пулинг состояний по схожести: на основе скрытых состояний элементов кодировщика, называемых **векторами-ключами $𝐾$**, скрытого состояния текущего элемента декодировщика, называемого **вектором-запросом $𝑞$**, и векторных представлений элементов целевой последовательности – **векторов-значений $𝑉$**, формируется контекстный вектор
как взвешенная сумма векторов-значений с такими весами, что чем больше соответствующий вектор-ключ схож с вектором-запросом, тем больше вес.

## Sequence-to-Sequence with RNNs and Attention mechanism

**Модель внимания — сходство входного и выходного состояния.**

$\large a(h, h^{'})$ — функция сходства состояний входа $h$ и выхода $h^{'}$

$\large a_{ti}$  — важность входа $i$ для выхода $t$ (attention score), $\large \sum_{i=1}a_{ti} = 1$

$\large c_t$ — вектор входного контекста для выхода t (context vector)

$\large h_i = f_{in}(x_i, h_{i-1});$

$\large \color{red}{\alpha_{ti} = norm_i \ a(h_i, h'_{t-1});}$

$\large \color{red}{c_t = \sum\limits_i \alpha_{t_i} h_i;}$

$\large h'_t = f_{out}(h'_{t-1}, y_{t-1}, \color{red}{c_t});$

$\large y_t = f_y(h'_t, y_{t-1}, \color{red}{c_t}).$

Примечание: $\large \displaystyle \color{red}{norm_i(p_i)} = {p_i \over \sum\limits_k p_k} .$

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq2seq_encoder_decoder_with_attention.png" width="700">

[К.В. Воронцов, Машинное обучение: Обработка последовательностей и модели внимания](http://www.machinelearning.ru/wiki/images/1/19/Voron-ML-Attention-slides.pdf)

Особенности:

* можно отказаться от рекуррентности по $h_i$;

* можно вводить обучаемые параметры в $a$ и $c$.

Как решить проблему “бутылочного горлышка”?
* формировать свой контекст $c_t$ для каждого элемента **выходной последовательности** $y_t$,
* использовать для формирования контекста $c_t$ все **скрытые состояния** кодировщика $h_i$.

Для формирования **векторов контекста** $(c_1, ..., c_T)$ возьмем линейную комбинацию **скрытых состояний** кодировщика $h_i$ с весами $a_{ti}$:
$$ c_t=\sum_{i=1}^{N}a_{ti}h_i.$$

$\alpha_{ti}$ называются **весами внимания**.

Веса $a_{ti}$ указывают, какие **скрытые состояния** кодировщика $h_i$ важны для формирования элемента **выходной последовательности** $y_t$.  Они “показывают” декодировщику куда “смотреть” при генерации данного элемента. Такой механизм в нейросетях получил название **attention** (внимание).

Веса $a_{ti}$ предсказывает сама модель. Для удобства веса подбираются таким образом, чтобы их сумма для каждого **вектора контекста** $c_t$ была равна 1 (нормализация):
$$ \sum_{i=1}^{N}a_{ti} = 1,$$

$$  0\leqslant a_{ti} \leqslant 1.$$

Для этого на выходе предсказывающего веса слоя ставят **SoftMax**.

Чтобы **вектор контекста** $c_t$ содержал информацию об уже сгенеренных элементах **выходной последовательности**, значение веса до нормализации  $e_{ti}$ зависит не только от скрытого состояния кодировщика $h_i$, но и от предыдущего скрытого состояния декодировщика $s_{t-1}$.

## RNN + Attention

В прошлой лекции мы работали с RNN без внимания. Теперь посмотрим, как это будет со слоем Attention.

Материал на основе [официальной документации PyTorch](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

### Подготовка данных

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Нам понадобится уникальный индекс для каждого слова, чтобы позже использовать его в качестве входных данных и таргетов. Сделаем вспомогательный класс Lang, из словарей **слово → индекс** (word2index) **и индекс → слово** (index2word), а также счетчик каждого слова word2count, который будет использоваться для замены редких слов позже.

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Все файлы представлены в формате Unicode, поэтому для упрощения мы преобразуем символы Unicode в ASCII, сделаем все строчными и уберём большую часть знаков препинания.

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

Делим файл на строки, а строки на пары.

In [ ]:
!wget https://github.com/L1aoXingyu/seq2seq-translation/blob/master/data/eng-fra.txt


In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Для скорости сократим датасет до предложений не длинее 10 слов, и отфильтруем апострофы.

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

### Кодировщик-декодировщик

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

На каждом этапе декодирования декодеру предоставляется входной токен и скрытое состояние. Начальный входной токен — токен начала строки <SOS>, первое скрытое состояние — вектор контекста (последнее скрытое состояние кодировщика).

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

### Слой Attention

Сначала мы вычисляем **набор весов Attention**. Они будут умножены на выходные векторы кодера для создания взвешенной комбинации. Результат **`attn_applied`** должен содержать информацию об этой конкретной части входной последовательности и, таким образом, помогать декодеру выбирать правильные выходные слова.

Вычисление весов Attention выполняется с помощью линейного слоя **`attn`**, В данных присутствуют предложения всех размеров, для фактического создания и обучения этого слоя нужно выбрать максимальную длину предложения. В предложениях максимальной длины будут использоваться все веса внимания, в то время как в более коротких предложениях будут использоваться только первые несколько.

Здесь реализован механизм **аддитивного внимания**, [Bandanau et al.](https://arxiv.org/abs/1409.0473). Этот механизм использует изученную модель выравнивания для вычисления оценок внимания между скрытыми состояниями кодера и декодера и полносвязную сеть для расчета весов Attention.

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

### Подготовка

**Подготовка данных**

Для обучения для каждой пары нам понадобится входной тензор (индексы слов во входном предложении) и целевой тензор (индексы слов в целевом предложении). При создании этих векторов мы добавим токен EOS к обеим последовательностям.

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

**Обучение модели**

Для обучения мы пропускаем предложение через кодировщик и отслеживаем каждый выход и последнее скрытое состояние. Затем декодер получает токен <SOS> в качестве первого входа и последнее скрытое состояние кодировщика в качестве первого скрытого состояния.

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

**Процесс обучения**:

* Запустить таймер
* Инициализировать оптимизаторы и loss
* Создайтnm набор обучающих пар

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Функция для построения графиков.

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

**Тестирование**

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

Мы можем оценить случайные предложения из обучающего набора:

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### Обучение и тестирование

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

### Визуализация Attention

In [ ]:
%matplotlib inline
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])

evaluateAndShowAttention('il n est pas aussi grand que son pere')
# evaluateAndShowAttention('je suis trop fatigue pour conduire')
# evaluateAndShowAttention('je suis desole si c est une question idiote')

## Проблема attention

Attention решает проблему "забывания" при работе с последовательностями. Но цена этого решения — квадратичное возрастание вычислительной сложности с ростом длины последовательности.

Вычислительная сложность **одного слоя RNN** составляет $O(bn d^2)$, где $b$ — длина батча, $n$ — число токенов и $d$ — размерность входа. Часть $d^2$ обусловлена матричным перемножением внутри блока RNN.

Вычислительная сложность **одного слоя attention** в простейшей реализации составляет $O(bn^2 d)$, то есть растет квадратично при росте длины последовательности $n$. Это объясняется тем, что длина выходной последовательности приблизительно равна длине входной последовательности $n$, и необходимо для каждого выходного токена рассчитать коэффициенты attention со всеми входными токенами. Сложность расчета одного коэффициента в простейшем случае составляет $O(d)$.

**Ни рекуррентные сети, ни attention не могут эффективно работать с очень длинными последовательностями.** RNN/LSTM "забывают" начало последовательности, а attention просто не может выполнить расчет за разумное время.

## Разновидности функций сходства векторов

$\large a(h, h') = h^Th'$ — скалярное произведение;

$\large a(h, h') = exp(h^Th')$ — тогда norm превращается в SoftMax;

$\large a(h, h') = h^T\color{red}{W}h'$ — c матрицей обучаемых параметров $\color{red}{W}$;

$\large a(h, h') = \color{red}{w}^Tth(\color{red}{U}h + \color{red}{V}h')$ — аддитивное внимание с $\color{red}{w, U, V}$.

Вводя внимание, мы говорили о некоторой **функции сходства** между скрытым состоянием декодировщика $h'$ и скрытым состоянием кодировщика $h$. Обобщением механизма внимания является введение в функцию сходства  **обучаемых параметров**.

Какие вообще бывают функции сходства?


1.   Первое, что приходит голову — просто считать скалярное произведение $h$ и $h'$.
2.   Также можно брать от него экспоненту, тогда оператор нормировки превращается в **SoftMax**.

Первые два способа возможны, только если потребовать, чтобы $h$ и $h'$ имели одинаковую размерность.

3.   Можно вводить матрицу обучаемых параметров $W$.
4.   Можно вводить небольшую двухслойную нейронную сеть с несколькими весовыми матрицами. Такое введение функции сходства называется аддитивным вниманием.



## Key, query, value

**Линейные преобразования векторов** **Query**, **Key** и **Value**.

Наиболее часто используемым подходом является введение трех типов векторов, которые называют **Query**, **Key** и **Value**.

Для каждого типа вектора вводится свое линейное преобразование, которое из исходного вектора делает вектор в каком-то другом пространстве. Все три они
обычно приводятся к одной и той же размерности, обозначенной $d$.

$\large a(h_i, h^\prime_{t-1}) = (\color{red}{W_k}h_i)^T(\color{red}{W_q}h^\prime_{t-1}) / \sqrt d$

$\large \alpha_{ti} = SoftMax_i \space a(h_i, h^\prime_{t-1})$

$\large c_t = \Sigma_i \alpha_{ti} \color{red}{W_v} h_i$

$ \large \color{red}{W_q}_{d \times dim(h^\prime)}, \color{red}{W_k}_{d \times dim(h)}, \color{red}{W_v}_{d \times dim(h)}$ — матрицы весов линейных нейронов (обучаемые линейные преобразования в пространство размерности $\large d$).

Возможно упрощение модели: $\large \color{red}{W_k} \equiv \color{red}{W_v}$

Функция сходства $a$ — это скалярное произведение, но перед тем,
как сделать скалярное произведение, каждый из двух векторов $h$ и $h'$ мы переводим с помощью линейного преобразования в новое пространство. Тем самым мы даем возможность модели обучить эти параметры матриц весовых коэффициентов $W_k$ и $W_q$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/query_key_value.png" width="250">

<em>Source: <a href="http://www.machinelearning.ru/wiki/images/1/19/Voron-ML-Attention-slides.pdf">Обработка последовательностей: модели внимания и трансформеры</a></em>

Два преобразованных вектора они скалярное произведение, которое для нормировки делится на корень из размерности.

Нормировка на корень из размерности позволяет сделать SoftMax более сглаженным. Беда может заключаться в том, что в результате скалярного произведения могут образовываться слишком большие или слишком маленькие значения, соответственно, будет слишком большой разброс, и когда мы будем считать экспоненту, получим вектор, который стремится к нулям или к единицам.

Чтобы этого не происходило, производится деление на корень из размерности. Идея в том, что скалярное произведение двух векторов в пространстве размерности $d$ — это сумма $d$ компонент. Закон больших чисел говорит о том, что когда мы складываем много одинаково распределенных случайных
величин, то их дисперсия растет пропорционально $d$ и, соответственно,
среднеквадратическое отклонение — это $\sqrt d$.

Далее мы к результату нормированного скалярного произведения применяем SoftMax.

Еще одно обобщение: когда мы считаем вектор контекста, то мы складываем не сами входные векторы $h_i$, а преобразуем их в векторы значений (Value).

У нас есть три места, где используются векторы, и в каждом из этих мест выполняется свое преобразование вектора в пространство размерности $d$.

Матрицы весов этих преобразований обучаются вместе со всей остальной моделью.

Иногда вводят упрощение и считают преобразования Key и Value одним преобразованием. Это позволяет сократить количество параметров.

### Softmax normalization

Вы могли заметить, что в формуле для вычисления сходства между Key и Query мы делим на $\sqrt d$. Давайте убедимся, что стандартное отклонение скалярного произведения двух величин хорошо оценивается корнем из размерности.



Давайте сгенерируем вектор из многомерного нормального распределения

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = np.random.normal(0, 100, size=(10000))

plt.title("Normal distribution, std = 100")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")

plt.hist(a)
plt.show()

И посмотрим, что будет с распределением значений этого вектора, если к нему применить SoftMax

In [ ]:
from scipy.special import softmax

plt.title("Softmax on N(0, 100)")
plt.ylabel("Softmax value")
plt.xlabel("Sample index")

plt.plot(softmax(a))
plt.show()

Почти все значения 0 и одно (наибольшее), стало 1.
Мы получили так называемое вырожденное распределение — у него вся плотность сосредоточена в одной точке.

И получили на абсолютно случайных данных.

Это приведет к затуханию градиента: мы будем распространять ошибку только для 1 значения из 10000. Учиться сеть будет плохо.

Но этого можно избежать — давайте просто стандартизируем наши данные:

In [ ]:
std = np.random.normal(0, 100, size=(10000))

unit_std = std / 100

plt.title("Normal distribution, std = 100")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")
plt.hist(std)
plt.show()

plt.title("Normal distribution, std = 1")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")
plt.hist(unit_std)
plt.show()

По сути, в распределении ничего не поменялось, только масштаб. Но теперь SoftMax работает нормально

In [ ]:
plt.title("Softmax on N(0, 1)")
plt.ylabel("Softmax value")
plt.xlabel("Sample index")

plt.plot(softmax(unit_std))
plt.show()

Остается только понять, как нормировать наши данные в нашем слое. Считать налету, наверное, не лучшая идея.

Наш слой делает скалярное произведение между двумя векторами, предположим, нормально распределенными.

In [ ]:
def statistics(dimensionality, experiments=int(10e4)):
    c = []
    for i in range(experiments):
        a = torch.normal(0, 1, size=(int(dimensionality),))
        b = torch.normal(0, 1, size=(int(dimensionality),))
        c.append(torch.dot(a, b))

    c = torch.Tensor(c)
    return float(c.mean()), float(c.std())

In [ ]:
import torch

means, stds = {}, {}
dims = torch.linspace(0, 100, 20)

for dim in dims:
    dim = float(dim)
    t_mean, t_std = statistics(dim)
    means[dim] = t_mean
    stds[dim] = t_std

In [ ]:
x = list(means.keys())
y = list(means.values())

plt.plot(x, y)
plt.axhline(y=0, c="r", linestyle="--")
plt.legend(["Mean value", "Mean = 0"])
plt.title("Mean value of dot products")
plt.ylabel("Mean value")
plt.xlabel("Vector dimensionality")
plt.show()

Видим, что среднее не сильно отличается от 0 (можно показать, что в среднем оно равно 0 для произведения нормально распределенных величин).

А вот стандартное отклонение растет. И можно предположить, что растет оно как корень из размерности вектора. Так и есть. Потому и появляется именно такой нормировочный множитель в attention

In [ ]:
x = list(stds.keys())
y = list(stds.values())
plt.scatter(x, y)
plt.title("Std values and square distance")
plt.xlabel("Vector dimensionality")

x = np.linspace(0, 100, 10000)
plt.plot(x, x**0.5, color="r")
plt.legend(["sqrt(x)", "empirical std"])
plt.show()

## Multihead Attention

**Идея:** $J$ разных моделей внимания совместно обучаются выделять различные аспекты входной информации (например, части речи, синтаксим, фразеологизмы):

$\large c_j = Attn(\color{red}{W^j_q}q, \color{red}{W^j_k}H,\color{red}{W^j_v}H, \ j = 1, \dots, j)$

**Варианты** агрегирования выходного вектор:

$\large \displaystyle c = {1 \over j} \sum\limits^J_{j=1}c^j$ — усреднение;

$\large \displaystyle c = [c^1 \dots c^J]$ — конкатенация;

$\large \displaystyle c = [c^1 \dots c^J]\color{red}{W}$ — возвращение к нужной размерности.

Давайте посмотрим на то, к каким словам предложения gave может иметь отношение. В общем случае глагол может иметь связку со многими частями предложения. Как, например, с подлежащим, так и с причастиями.

В идеале, нам бы хотелось обратить внимание функции (attention) на все эти взаимосвязи. Для этого нам просто надо поставить несколько attention слоев параллельно. Тогда каждый из них будет учить что-нибудь свое по аналогии со сверточными слоями.

* Чтобы осуществить задуманное, вместо одного набора query будем использовать несколько независимых наборов.

* Причем каждый набор будет считаться уникальной матрицей.

* Аналогично сделаем для keys и values. Количество таких наборов внутри keys, queries, values должно быть **одинаковым**.

* Обозначим это число как $J$, далее производим аналогичные манипуляции, при этом введем в параллель h таких функций attention.

* На последнем шаге мы их соединяем (конкатинируем).

* При этом можно заметить, что при таком подходе на каждом шаге размерность токена будет увеличиваться (если, например, в качестве и key, и value, и query мы подаем одно и тоже предсталение токена). Если хотим сохранять управление размерностью токена, то придется получать по меньшей мере value путем домножения на матрицу, размерность которой по второй оси меньше — **выполнять проекцию наших токенов в пространство меньшей размерности**.

* В частности, можно подобрать размерность этого пространства таким образом, чтобы при конкатенации размерность полученного токена равнялась исходной.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/multihead_self_attention_layer.png" width="700">

# Transformer для машинного перевода

## Архитектура сети Transformer

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/transformer_architecture.png" width="450">

<em>Архитектура трансформера</em>

<em>Source: <a href="https://arxiv.org/pdf/1706.03762.pdf"> Attention Is All You Need</a></em>

## Общий пайплайн задачи машинного перевода

Трансформер (англ. transformer) — это нейросетевая архитектура на основе моделей внимания и полносвязных слоёв, без RNN.

**Схема преобразований данных в машинном переводе:**

$S = (w_1, \dots , w_n)$ — слова предложения на входном языке

$\color{blue}{\downarrow \quad \text{обучаемая или предобученная векторизация слов}}$

$X = (x_1, \dots , x_n)$ — эмбеддинги слов входного предложения

$\color{blue}{\downarrow \quad \text{трансформер-кодировщик}}$

$Z = (z_1, \dots , z_n)$ — контекстные эмбеддинги слов

$\color{blue}{\downarrow \quad \text{трансформер-декодировщик, похож на кодировщика}}$

$Y = (y_1, \dots , y_m)$ — эмбеддинги слов выходного предложения

$\color{blue}{\downarrow \quad \text{генерация слов из построенной языковой модели}}$

$\tilde S = (\tilde w_1, \dots , \tilde w_m)$ — слова предложения на выходном языке


## Архитектура трансформера-кодировщика

Порядок вычислений трансформера-кодировщика:

1. Добавляются позиционные векторы $p_i$:

$\qquad \large h_i = x_i + p_i;$

$\qquad \large H = (h_1, \dots, h_n).$

$\qquad$ Размерность: $dim \ x_i, \ p_i, \ h_i = 512, \ dim \ H = 512 \times n$

2. Многомерное самовнимание:

$\qquad \large h^j_i = Attn(\color{red}{W^j_q}h_i, \color{red}{W^j_k}H, \color{red}{W^j_v}H).$

$\qquad$ Размерность: $j = 1, \dots, J=8, \ dim \ h^j_i = 64, \ dim \ W^j_q, \ W^j_k, \ W^j_k = 64 \times 512 $

3. Конкатенация:

$\qquad \large h'_i =  MH_j (h^j_i) \equiv [h^1_i, \dots, h^J_i].$

$\qquad$ Размерность: $dim \ h'_i = 512$

4. Сквозная связка + нормировка уровня:

$\qquad \large h''_i =  LN(h'_i + h_i; \color{red}{\mu_1, \sigma_1}).$

$\qquad$ Размерность: $dim \ h''_i, \ \mu_1, \ \sigma_1 = 512$

5. Полносвязная 2-хслойная сеть FFN:

$\qquad \large h'''_i = \color{red}{W_2}ReLU(\color{red}{W_1}h''_i + \color{red}{b_1}) + \color{red}{b_2}.$

$\qquad$ Размерность: $dim \ W_1 = 2048\times512, \ dim \ W_2 = 512\times2048$

6. Сквозная связь + нормировка уровня:

$\qquad \large z_i = LN(h'''_i + h''_i; \color{red}{\mu_2, \sigma_2}).$

$\qquad$ Размерность: $dim \ z_i, \ \mu_2, \ \sigma_2 = 512$

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/transformer_encoder.png" width="200">

<em>Архитектура трансформера-кодировщика</em>

<em>Source: <a href="http://www.machinelearning.ru/wiki/images/1/19/Voron-ML-Attention-slides.pdf"> К.В. Воронцов, Машинное обучение: Обработка последовательностей и модели внимания</a></em>

Особенности архитектуры трансформера-кодировщика:

* вычисления параллельны по элементам последовательности $(x_1, \dots, x_n) \rightarrow (z_1,\dots, z_n)$, что было бы невозможным в RNN;

* N = 6 блоков $h_i \rightarrow \Box \rightarrow z_i$ соединяются последовательно;

* возможно использование предварительно обученных ембеддингов $x_i$;

* возможно обучение эмбеддингов $x_i \in \mathbb{R}^d$ слов $w_i \in V$:

$\qquad \large x_i = \color{red}{u_{w_i}}$ или в матричной записи $X_{d \times n} = \color{red}{U_{d \times V}} \cdot B_{V \times n}$, где:

$\qquad V$ — словарь слов входных последовательностей,

$\qquad \color{red}{U}$ — матрица обучаемых векторных представлений слов,

$\qquad b_{vi} = [w_i = v]$ — матрица бинарного (one-hot) кодирования;

* нормировка уровня (Layer Normalization):

$\qquad  \large x_i, \ \color{red}{\mu}, \ \color{red}{\sigma} \in \mathbb{R};$

$\qquad  \large \displaystyle LN_s(x; \color{red}{\mu}, \ \color{red}{\sigma}) = \color{red}{\sigma_s} {{x_s - \overline x} \over \sigma_x} + \color{red}{\mu_s}, \ s = 1, \dots, d;$

$\qquad \displaystyle \overline x = {1 \over d} \sum\limits_{s}x_s$ и $\displaystyle \sigma^2_x = {1 \over d} \sum\limits_{s}(x_s - \overline x)^2$ — среднее и дисперсия $x$.

### Positional encoding

Единственный возможный минус — нейросеть не учитывает порядок слов в предложении при составлении embedding. Это может нам мешать. Например, если в предложении два it, то они часто относятся к разным словам. Поэтому хотелось бы уметь учитывать информацию о позиции. Для этого к $X$ при составлении $Q$ добавляется информация о позиции.

Делается это хитрым образом: мы добавляем к каждому значению исходного вектора токенов некую комбинацию $sin$ и $cos$ с разными параметрами. **Значения суммируются, а не конкатенируются.**

Вектор $PE$, который мы будем добавлять к $X$, будет определяться по следующей формуле:

$$p_{pos, 2i} = \sin \left({\dfrac {pos} {10000^{2i/d}}}\right)$$

$$p_{pos, 2i+1} = \cos \left({\dfrac {pos} {10000^{2i/d}}}\right)$$

$pos$ &mdash; это позиция токена

$d$ &mdash; количество размерностей токена

$i$ &mdash; $i$-тая размерность токена

In [ ]:
import math
import torch


class PositionalEncoding(torch.nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].detach()
        return x

In [ ]:
pe = PositionalEncoding(20)
y = pe(
    torch.zeros(1, 100, 20)
)  # sequence of shape 100, every token of sequence has shape 20

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 0:4].data.numpy())
plt.legend(["dim %d" % p for p in [1, 2, 3, 4]])
plt.show()

В результате каждая позиция кодируется уникальным представлением. При этом представление позволяет легко находить слова на заданном расстоянии от исходного (у них будет одинаково значение сигнала по какой-то оси).

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
plt.legend(["dim %d" % p for p in [4, 5, 6, 7]])
plt.show()

Это помогает трансформеру достаточно уникальным образом определять каждую позицию и понимать относительное расстояние между разными токенами.

## Архитектура трансформера-декодировщика

Авторегрессионный синтез последовательности:

$\large y_0 = \langle {BOS} \rangle$ — эмбеддинг символа начала.

Для всех $t = 1, 2, \dots$ выполняется следующая последовательность вычислений:

1. Маскирование "данных из будущего":

$\qquad \large h_t = y_{t-1} + p_t;$

$\qquad \large H_t = (h_1, \dots, h_t).$

2. Многомерное самовнимание:

$\qquad \large h'_t = LN \circ MH_j \circ Attn(\color{red}{W^j_q}h_t, \color{red}{W^j_k}H_t, \color{red}{W^j_v}H_t).$

3. Многомерное внимание на кодировку $Z$:

$\qquad \large h''_t = LN \circ MH_j \circ Attn(\color{red}{W^j_q}h'_t, \color{red}{W^j_k}Z, \color{red}{W^j_v}Z).$

4. Двухслойная полносвязная сеть:

$\qquad \large y_t = LN \circ FFN(h''_t).$

5. Линейный предсказывающий слой:

$\qquad \large p(\tilde w | t) SoftMax_{\tilde w}(\color{red}{W_y}y_t + b_y).$

Генерация $\tilde w_t = argmax(p(\tilde w | t))$ продолжается пока $\tilde w_t \neq \langle {EOS} \rangle$.


<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/transformer_decoder.png" width="350">

<em>Архитектура трансформера-декодировщика</em>

<em>Source: <a href="http://www.machinelearning.ru/wiki/images/1/19/Voron-ML-Attention-slides.pdf"> К.В. Воронцов, Машинное обучение: Обработка последовательностей и модели внимания</a></em>

## Особенности обучения

Transformer обладает большим количеством обучаемых параметров и
оказывается достаточно чувствительным к выбору learning rate и batch
size.

<img src ="https://imageup.ru/img183/4513756/2023-09-07_07-40.png" width="600">

<em>Влияние learning rate на качество</em>

<em>Source: <a href="http://dx.doi.org/10.2478/pralin-2018-0002"> Popel Martin, Bojar Ondˇrej. Training Tips for the Transformer Model
</a></em>

**Большая скорость обучения приводит к лучшему результату.**

<img src ="https://imageup.ru/img47/4513761/2023-09-07_07-44.png" width="600">

<em>Влияние batch size на качество</em>

<em>Source: <a href="http://dx.doi.org/10.2478/pralin-2018-0002"> Popel Martin, Bojar Ondˇrej. Training Tips for the Transformer Model
</a></em>

**Больший batch size приводит к повышению качества**.

При обучении таких сложных архитектур, как трансформер, очень важно подобрать гиперпараметры, при этом хорошие значения как для LR, так и для BS лежат в достаточно большом диапозоне.



## Данные и промты

Большие языковые модели обучаются в 4 этапа:

* (1) Предварительное обучение без учителя на основе необработанного текста для изучения представлений общего назначения

* (2) Fine-Tuning на промтах

* (3) Оценка ответов нейросети и настройка инструкций человеком

* (4) Обучение с подкреплением для лучшего соответствия конечным задачам и предпочтениям пользователя.

<img src ="https://imageup.ru/img282/4513817/2023-09-07_08-01.png" width="900">

<em>Дообучение с учителем и обучение с подкреплением на основе отзывов людей</em>

<em>Source: <a href="https://arxiv.org/pdf/2203.02155.pdf"> Training language models to follow instructions with human feedback
</a></em>

Согласно последним исследованиям, большее количество данных вовсе не обязательно приводит к повышению качества. В статье [LIMA: Less Is More for Alignment](https://arxiv.org/pdf/2305.11206.pdf) делается вывод, что **достаточно 1000** хорошо подобранных **подсказок и ответов**, **без** какого-либо **обучения с подкреплением** или моделирования человеческих предпочтений.

Почти все знания в больших языковых моделях усваиваются во время предварительного обучения, и для обучения моделей производству высококачественного вывода необходимы лишь ограниченные данные по настройке инструкций.

**Как писать промты?**

[Обшириный гайд для ChatGPT-like моделей](https://www.greataiprompts.com/chat-gpt/how-to-write-better-chat-gpt-or-gpt-prompts/)

1. Определите задачу и цель вашего запроса

    Прежде чем приступить к созданию подсказки, очень важно знать, какую задачу вы хотите, чтобы модель выполняла, и каким должен быть желаемый результат.

    Вы заинтересованы в приобретении знаний, повествовании или другом виде художественного самовыражения?

2. Будьте конкретны в своем письме

    Дайте конкретные инструкции или попросите конкретную информацию вместо того, чтобы задавать общий вопрос.

3. Используйте понятный язык

    Не используйте жаргон или слова, которые можно неоднозначно понять.

4. Предоставьте контекст

    Предоставьте модели достаточно справочной информации, чтобы помочь ей понять тему или сценарий, который вы представляете.

5. Учитывайте длину

    Обширная подсказка может дать больше контекста и подробностей, но модели будет труднее ее понять.

6. Тестирование

    Если результаты не соответствуют вашим ожиданиям, внесите изменения в подсказку и запустите тест еще раз.

**Пример аккуратно написанных подсказок**

* «Я хочу, чтобы вы действовали как терминал Linux. Я буду вводить команды, а вы ответите тем, что должен показать терминал. Я хочу, чтобы вы ответили только выводом терминала внутри одного уникального блока кода».

* Есть ли в тексте упоминание стратегий?

**Примеры плохо написанной подсказки**
* «Взаимодействуй со мной, как как терминал Linux».

* Есть ли в тексте упоминание стратегий экономического развития?

### Галлюцинации

Одной из ключевых проблем LLM является придумывание ответов, синтаксически похожих на настоящие, однако являющихся бредом.

<img src ="https://medpsy.world/wp-content/uploads/2020/01/Naoto-Hattori.jpg" width="1000">


<em>Naoto Hattori</em>





Согласно

### Masked Self-Attention Layer


Допустим,  у нас стоит проблема, что мы не должны видеть часть слов в предложении — например, при генерации текста (по текущим словам предсказать следующее). Например, хотим сгенерировать фразу "robot must obey orders" на основе только первого слова.

Если мы подадим в нейронную сеть во время обучения сразу все предложение, которое хотим генерировать, то у нас в первых словах будет "протекать" информация о предыдущих.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/queries_keys_scores_before_softmax.png" width="800">

В этом случае мы можем просто на соответствующих местах матрицы $E$ поставить минус бесконечности. Тогда в эмбеддингах слов, которые не должны знать о каких-то словах, информации об этих словах не будет.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/scores_before_softmax_apply_attention_mask_masked_scores_before_softmax.png" width="800">

В результате после SoftMax "лишняя" информация не будет использоваться при генерации ответа на query.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/masked_scores_softmax_along_rows_scores.png" width="800">

<em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em>

Благодаря этому трюку у нас получается обучать transfomer по-прежнему как простую single-pass нейросеть, а не "скатываться" в RNN, где у нас возникнут проблемы с градиентами и временем работы.

## Небольшая историческая справка

Далее часть текста основана на статье [GPT для чайников: от токенизации до файнтюнинга](https://habr.com/ru/articles/599673/).

Качественный скачок в решении NLP-задачи произошёл благодаря рекуррентным сетям. Затем появился механизм внимания — **attention**, который применялся в RNN и давал огромный прирост качества.

Далее статья _Attention Is All You Need_ показала, что attention отлично работет вовсе без RNN. В этой статье трансформер из кодировщика и декодировщика был обучен переводить текст, и делал это великолепно.

Затем произошёл раскол: в OpenAI решили сконцентироваться на декодерах, а в Google — на энкодерах. Так появились первые **GPT** и **BERT**.

**Так что же такое GPT?**
* Это нейронная сеть для генерации (продолжения) текста.

* Более строго — языковая модель, основанная на архитектуре трансформер и обученная в self-supervised режиме на огромном [корпусе](https://philology.by/about/yaskevich/corpus-linguistics-yaskevich) текстовых данных.

**Оригинальные статьи про поколения GPT:**
* [Improving Language Understanding by Generative Pre-Training (2018)](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
* [Language Models are Unsupervised Multitask Learners (2019)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
* [Language Models are Few-Shot Learners (2020)](https://arxiv.org/pdf/2005.14165.pdf)

## Hugging Face

Для работы с GPT будем использовать предобученную модель. Лучший выбор для работы с трансформерами — библиотеки от **Hugging Face**: `transformers`, `tokenizers`, `datasets`.

Hugging Face занимается стандартизацией применения трансформеров, а также хранит наборы весов и датасеты для различных NLP-задач. Воспользуемся русскоязычной моделью ruGPT3 и дообучим её.



Установим библиотеку Transformers:

In [ ]:
!pip install -q transformers

Выберем необходимую модель. API для различных моделей одинаковый, для подмены модели достаточно изменить название модели `model_name`.

In [ ]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from IPython.display import clear_output

transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading and initialization of model and tokenizer
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

clear_output()

## Языковое моделирование

**Языковое моделирование** — предсказание следующего слова (или части слова) с учётом предыдущего контекста.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/yandex_search.png" width="700"></center>

<center><em>Source: <a href="https://habr.com/ru/articles/599673/">GPT для чайников: от токенизации до файнтюнинга</a></em></center>

Для того, чтобы сгенерированное моделью продолжение текста было верным не только грамматически, но и семантически, модель должна хорошо понимать смысл изначального текста и, желательно, даже иметь знания о реальном мире.

Эти внутренние знания позволяют модели отвечать на вопросы, суммаризировать текст, создавать диалоговые системы и многое другое.

Например, если мы хотим при помощи языковой модели ответить на вопрос: **«Сколько будет 2+2?»**, то можем подать на вход модели следующий текст:\
`«Вопрос: Сколько будет 2+2? Ответ: … »`\
и естественным продолжением такого текста будет ответ на вопрос, поэтому модель допишет `«4»`

In [ ]:
text = "Вопрос: 'Сколько будет 2+2?'\nОтвет:"
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
out = model.generate(input_ids, do_sample=False, max_length=20, pad_token_id=20)

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Похожим способом можно кратко пересказывать тексты, если в конце дописывать `«TL:DR»`, т.к. модель во время обучения запомнила, что после этих символов идёт краткое содержание. Подбор модификаций текста называется **«Prompt Engineering»**. Такая простая идея позволяет решать практически неограниченное количество задач. Именно поэтому многие считают GPT-3 подобием сильного искусственного интеллекта.

# Как работает GPT

## Токенизация

Один из ключевых этапов в обработке текста — **токенизация**. На этом этапе происходит разделение текста на отдельные единицы — предложения и слова. Затем создается словарь, в который заносятся уникальные лексемы, встретившиеся в корпусе или тексте. На этих этапах можно столкнуться с несколькими проблемами.

**Проблема 1. Размер словаря**

Самый простой способ токенизации — назначить каждому уникальному слову своё число. Но есть проблема: слов и их форм миллионы, и поэтому словарь таких слов получится чересчур большим, а это будет затруднять обучение модели.

Можно разбивать текст не на слова, а на отдельные буквы (char-level tokenization), тогда в словаре будет всего несколько десятков токенов, НО в таком случае уже сам текст после токенизации будет слишком длинным, а это тоже затрудняет обучение.

**Проблема 2. Богатая морфология**

"Нейросеть", "сетка", "сеть" являются разными словами, но имеют схожий смысл. Эту проблему классически всегда решал этап **стемминга** (удаление суффикса, приставки, окончания) или **лемматизации** (приведение слова к канонической форме).

**Проблема 3. Сложные слова**

Но все проблемы эти этапы не решают. В германских языках (в английском, немецком, шведском и т.д.) очень продуктивно образуются новые сложные слова. Значения таких слов выводятся из значения их элементов. Их можно создавать бесконечно долго, и большинство из них не зафиксировано в «бумажном» словаре.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/swedish_word_example.png" width="600"></center>

<center><em>Пример шведского названия гаечного ключа для колеса мотоцикла</a></em></center>

<center><em>Source: <a href="https://sysblok.ru/nlp/7250/">Как работает алгоритм токенизации текстов для нейросетей</a></em></center>

При работе с этими языками сложность также возникает на этапе составления словаря. При составлении словаря модели ориентируются на частотность (например, сохраняем слово, если оно встретилось чаще пяти раз), поэтому не будут запоминать такое длинное и сложное слово.

**Проблема 4: Границы слова**

Для нас, привыкших к языкам европейского типа, слово — это набор букв между пробелами и знаками препинания. Но в английском языке многие сложные слова пишутся раздельно, а в японском, наоборот, между словами вообще нет пробелов. Поэтому универсальный токенизатор создать было нелегко.

**Решение — Byte Pair Encoding**

Изначально алгоритм компрессии BPE позволяет моделям узнавать как можно больше слов при ограниченном объеме словаря.

1.   Слово = последовательность токенов
2.   Словарь = все токены
3.   Повторять, пока не достигли ограничения на размер словаря:

     Назначаем новым токеном объединение двух существующих токенов, которое
встречается чаще других пар в корпусе (встречаются вместе).

В применении BPE возможны разные варианты. Один из естественных – идём по всем токенам по убыванию частоты, находим соответствующую последовательность символов в корпусе, заменяем на токен.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/L08/subword_tokenization.png" width = "500"></center>

<center><em>Source: <a href="https://alexanderdyakonov.wordpress.com/2019/11/29/токенизация-на-подслова-subword-tokenization/">Токенизация на подслова (Subword Tokenization)</a></em></center>

Этот же способ помогает решить **проблему** **OOV (out of vocabulary)**. В обучающей выборке может не быть слова *Unfriendly*, но поскольку **Unfriendly** = **Un** + **friend** + **ly**, мы можем рассчитывать, что сеть будет правильно обрабатывать / генерировать и слово целиком.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/token_unfriendly.png" width="600"></center>

<center><em>Source: <a href="https://www.thoughtvector.io/blog/subword-tokenization/">Subword Tokenization — Handling Misspellings and Multilingual Data</a></em></center>

Но даже это иногда не самый оптимальный выбор. Чтобы сжать словарь ещё сильнее, для обучения GPT OpenAI использовали **byte-level BPE** токенизацию. Эта модификация BPE работает не с текстом, а напрямую с его байтовым представлением. Использование такого трюка позволило сжать словарь до всего-лишь ~50k токенов при том, что с его помощью всё ещё можно выразить любое слово на любом языке мира (и даже эмодзи).

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q transformers

In [ ]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from IPython.display import clear_output

transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading and initialization of model and tokenizer
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(device)

clear_output()

**Пример токенизации**

In [ ]:
text = "Нейронные сети - это очень просто и увлекательно"
tokens = tokenizer.encode(text, add_special_tokens=False)

decoded_tokens = [tokenizer.decode([token]) for token in tokens]

print("Original text:", text)
print("Tokens: ", tokens)
print("Decoded tokens: ", decoded_tokens)

Byte-level токенизатор **не гарантирует**, что для любого токена найдется **соответствующий** символ или слово. Некоторые **токены** **существуют** только **в комбинациях**. Так, представленные токены не декодируются по отдельности.

In [ ]:
print(tokenizer.decode([167]))
print(tokenizer.decode([245]))
print(tokenizer.decode([256]))

print(tokenizer.decode([167, 245, 256]))

## Архитектура GPT

При генерации продолжения текста с помощью GPT происходит следующее:

1. Входной текст токенизируется в последовательность чисел (токенов).
2. Список токенов проходит через Embedding layer (линейный слой) и преобразуется в список эмбеддингов.
3. К каждому эмбеддингу прибавляется **positional embedding**.
4. Список эмбеддингов проходит через несколько одинаковых блоков (Transformer Decoder Block).
5. После того, как список эмбеддингов пройдёт через последний блок, эмбеддинг, соответствующий последнему токену, матрично умножается на всё тот же входной, но уже транспонированный Embedding Layer, и после применения SoftMax получается распределение вероятностей следующего токена.
6. Из этого распределения выбирается следующий токен (например, с помощью argmax)
7. Полученный токен добавляется к входному списку токенов, шаги 1-6 повторяются

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/gpt3.gif" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/how-gpt3-works-visualizations-animations/">How GPT3 Works — Visualizations and Animations</a></em></center>

## Positional Encoding

В отличие от рекуррентных сетей, архитектура трансформера не чувствительна к порядку входных токенов, то есть при перемешевании слов местами выход будет получаться одинаковым (permutation invarience).

Позиционное кодирование описывает позицию объекта в последовательности так,  что каждой позиции соответствует уникальное представление.

**Почему не используется одно число, например значение индекса?**

Для длинных последовательностей индексы могут сильно увеличиваться по величине. Если вы нормализуете значение индекса так, чтобы оно лежало между $0$ и $1$, это может создать проблемы для последовательностей переменной длины, поскольку они будут нормализованы по-разному.

Поэтому в GPT используется кодирование позиции в виде вектора, который прибавляется к эмбеддингу токена. Эти позиционные эмбеддинги можно как зафиксировать заранее (так делается в оригинальном трансформере, см. пример на картинке), так и обучать, как в случае GPT.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/pos_encoding_visual.png" width="800"></center>

<center><em>Позиционные эмбеддинги оригинального трансформера</em></center>

<center><em>Source: <a href="https://habr.com/ru/articles/599673/">GPT для чайников: от токенизации до файнтюнинга</a></em></center>

Предположим, у вас есть входная последовательность длины $L$, и требуется задать положение $k$-того объекта в этой последовательности. Позиционное кодирование задается функциями синуса и косинуса различной частоты:

$$P(k, 2i) = sin (\frac{k}{2^{2i/d}})$$

$$P(k, 2i+1) = cos (\frac{k}{2^{2i/d}})$$

где $k$ — позиция объекта в последовательности, $\displaystyle 0\leq k< \frac{L}{2}$,

$d$ — размерность выходного пространства эмбеддингов,

$P(k,j)$ — функция, которая переводит позицию $k$ в индекс $(k,j)$ позиционной матрицы,

$n$ — константа, обычно равно $10 000$ согласно статье *Attention is all You Need*,

$i$ — индекс колонки, $0 \leq i < d/2$, одинаково для синуса и для косинуса.



## Transformer Decoder Block

Основной блок GPT состоит из слоёв self-attention, нормализации, feed-forward и residual connections.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_block.png" width="500"></center>

<center><em>Source: <a href="https://ai-news.ru/2019/06/obobshennye_yazykovye_modeli.html">Обобщенные Языковые Модели</a></em></center>


## Методы Генерации текста

Языковая модель генерирует распределение вероятностей следующего токена. Однако способы генерации текста могут отличаться. Далее разберём, какие варианты бывают.

Для наглядности применим основные методы для продолжения следующего текста  \
`'Определение: "Нейронная сеть" — это'`

In [ ]:
text = 'Определение: "Нейронная сеть" - это'
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

### Greedy Search

Очевидный вариант — ArgMax-генерация (жадный поиск). Выбирается максимально вероятный токен.

При таком способе мы не получим разнообразного текста на один и тот же запрос, и, что ещё хуже, генерация может застревать в локальных минимумах и выдавать повторяющиеся фрагменты, например `the the the the ...`.

In [ ]:
# ArgMax is defaulf behaviour
out = model.generate(input_ids, do_sample=False, max_length=30, pad_token_id=30)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

### Beam Search
Несколько более сложный и качественный способ сэмплирования — **beam search**. Каждый раз мы выбираем не один самый вероятный токен, а сразу несколько (`beam-size`), и дальше продолжаем поиск для каждого из выбранных токенов.

Таким образом создаётся **граф** со сгенерированными **вариантами предложений**. Далее выбирается предложение с наибольшей **perplexity** (уверенностью модели в реалистичности текста).

Обычно это приводит к высокой связности (когерентности) текста, но при этом к сухости и скучности текста. Также это не решает полностью проблему с повторениями кусочков текста.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/beam_search.png" width="500"></center>

<center><em>Source: <a href="https://habr.com/ru/articles/599673/">GPT для чайников: от токенизации до файнтюнинга</a></em></center>



In [ ]:
# Generation with beam-search
out = model.generate(input_ids, do_sample=False, num_beams=5, max_length=30, pad_token_id=30)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

### Сэмплирование с Температурой

Чтобы добавить тексту непредсказуемости и человечности, можно использовать вероятностное сэмплирование с температурой. Будет использоваться не самый вероятный токен, а случайный, с учётом распределения вероятностей.

Параметр температуры позволяет контролировать степень случайности. При нулевой температуре метод совпадает с жадным сэмплированием, при  большой температуре токены будут выбираться полностью случайно. Обычно хорошо работает температура в диапазоне `0.8–2.0`.

Формула модификации распределения вероятностей очень похожа на формулу распределения Больцмана: чем выше температура системы, тем больше "размазывается" распределение вероятностей её возможных состояний, отсюда слово "температура".

$$p=softmax(log(p)/t)$$

Стоит отметить, что случайная природа генерации будет иногда приводить к полностью некорректным результатам.

In [ ]:
out = model.generate(input_ids, do_sample=True, temperature=1.3, max_length=30, pad_token_id=30)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

### Сэмплирование с Ограничением Маловероятных Токенов (Nucleus sampling)

Можно ввести запрет на семплирование наименее вероятных токенов:

* `top-k` зануляет все вероятности, кроме $k$ наибольших;

* `top-p` оставляет минимальный набор токенов, причём сумма их вероятностей будет не больше $p$.

`top-p` ограничение называют **Nucleus Sampling**.

In [ ]:
out = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.3,
    top_k=20,
    top_p=0.8,
    max_length=30,
    pad_token_id=30
)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

## Сравнение поколений GPT


### GPT-1

**GPT — Generative Pretraining of Transformers**, состоящая всего из 12 слоёв и обученная на 7000 книгах. Хотя длинные тексты генерировались неважно, при файнтюнинге на узкоспециализированные задачи отрабатывала лучше SOTA.

Основным новшеством было доказательство того, что предобучение без учителя языковой модели позволяет добиться отличных результатов после дальшейшего файнтюнинга.

Максимальный размер контекста — 512 токенов.

### GPT-2

Вскоре после GPT-1 появился BERT, побивший её результаты. После чего OpenAI **увеличили количество слоёв в 10 раз** и довели **количество парамеров** до **1.5B**. Модель была дообучена на **8 миллионов сайтов**, суммарно на 40 Гб текста. Архитектрурно это та же модель с перемещёнными слоями нормализации.

GPT-2 научилась писать длинные связные тексты и даже решать при помощи prompt engineering множество новых задач.

Максимальный размер контекста — 1024 токенов

### GPT-3

Модель **увеличилась ещё в 10 раз (175B параметров)**, объём датасета — 570 Гб текста. Архитектурно та же, изменения коснулись оптимизации attention.

Теперь модель может писать рабочий программный код [(CODEX)](https://openai.com/blog/openai-codex/) и решать много других почти сверхъестественных задач ([воскрешать мёртвых ](https://futurism.com/openai-dead-fiancee)).

Максимальный размер контекста — 2048 токенов.

# Файнтюнинг

Воспользуемся моделью меньшего размера, чтобы она поместилась на GPU.

In [ ]:
!pip install -q transformers[torch]

In [ ]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from IPython.display import clear_output

transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

clear_output()

## Как происходит обучение

Обучающий текст нарезается на случайные блоки, которые составляются в последовательности из 1024 (2048 у GPT-3) токенов, разделяясь специальным `<|endoftext|>` символом. Во время обучения модель учится предсказывать (классифицировать) каждый токен в последовательности один за другим при помощи Cross-Entropy Loss.

Так как входная последовательность всегда заполнена до конца, padding не используется. Но во время инференса длина входного текста может быть произвольной, поэтому надо явно указывать, чем паддить оставшиеся позиции. По дефолту использутеся тот же `<|endoftext|>`.

В отдельных версиях GPT вышесказанное может модифицироваться. Например, в ruGPT3 гораздо больше специальных токенов: `<s\>`, `<s>`, `<pad>`, `<unk>`

## Обучающие данные
Будем учить GPT генерировать стихи Маяковского. В качестве обучающих данных возьмём всего лишь один стих.

In [ ]:
text = """Дым табачный воздух выел.
Комната —
глава в крученыховском аде.
Вспомни —
за этим окном
впервые
руки твои, исступленный, гладил.
Сегодня сидишь вот,
сердце в железе.
День еще —
выгонишь,
может быть, изругав.
В мутной передней долго не влезет
сломанная дрожью рука в рукав.
Выбегу,
тело в улицу брошу я.
Дикий,
обезумлюсь,
отчаяньем иссеча́сь.
Не надо этого,
дорогая,
хорошая,
дай простимся сейчас.
Все равно
любовь моя —
тяжкая гиря ведь —
висит на тебе,
куда ни бежала б.
Дай в последнем крике выреветь
горечь обиженных жалоб.
Если быка трудом уморят —
он уйдет,
разляжется в холодных водах.
Кроме любви твоей,
мне
нету моря,
а у любви твоей и плачем не вымолишь отдых.
Захочет покоя уставший слон —
царственный ляжет в опожаренном песке.
Кроме любви твоей,
мне
нету солнца,
а я и не знаю, где ты и с кем.
Если б так поэта измучила,
он
любимую на деньги б и славу выменял,
а мне
ни один не радостен звон,
кроме звона твоего любимого имени.
И в пролет не брошусь,
и не выпью яда,
и курок не смогу над виском нажать.
Надо мною,
кроме твоего взгляда,
не властно лезвие ни одного ножа.
Завтра забудешь,
что тебя короновал,
что душу цветущую любовью выжег,
и су́етных дней взметенный карнавал
растреплет страницы моих книжек…
Слов моих сухие листья ли
заставят остановиться,
жадно дыша?
Дай хоть
последней нежностью выстелить
твой уходящий шаг.."""

В библиотеке transformers есть готовые инструменты для подготовки датасета и даталодера. На вход нужен всего лишь один `.txt` файл с обучающим текстом.

In [ ]:
# Save text train data as .txt file
train_path = "train_dataset.txt"
with open(train_path, mode="w", encoding="utf-8") as f:
    f.write(text)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from warnings import simplefilter

simplefilter("ignore", category=FutureWarning)

# Creating Dataset
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)

# Сreating DataLoader (crop the text into optimal length pieces)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Training
Для файнтюнинга нам необходим объект класса Trainer, который сделает всю работу за нас. Далее нужно будет всего лишь запустить `trainer.train()`.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=200,  # number of training epochs
    per_device_train_batch_size=32,  # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_steps=10,  # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16,  # to make "virtual" batch size larger
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers=(
        torch.optim.AdamW(model.parameters(), lr=1e-5),
        None,
    ),  # Optimizer and learnig rate scheduler
)

In [ ]:
trainer.train()

## Результат файнтюнинга
Готово! Теперь давайте посмотрим, что же сочинит GPT в стиле Маяковского, если на вход подать такую строчку:

"Учим нейросеть за нейросетью!"

In [ ]:
# Probability sampling with limit example
text = "Как же сложно учить матанализ!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    out = model.generate(
        input_ids,
        do_sample=True,
        num_beams=2,
        temperature=1.5,
        top_p=0.9,
        max_length=100,
        pad_token_id=512
    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

**Полезные ссылки**
1. [GPT в картинках](https://habr.com/ru/post/490842/) — очень подробный разбор внутренней архитектуры GPT-2 с акцентом на иллюстрации.
2. [Трансформер в картинках](https://habr.com/ru/post/486358/) — очень подробный разбор архитектуры Transformer с акцентом на иллюстрации.
3. [Tokenizers tutorial](https://huggingface.co/docs/transformers/tokenizer_summary) — краткий разбор всех типов токенизаторов от Huggingface с примерами.
4. [Как генерировать текст](https://huggingface.co/blog/how-to-generate) — обзор способов сэмплирования текста с помощью языковых моделей (бимсёрч и тд).
5. [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) — оригинальная статья про первый трансформер.
6. [GPT-1](https://openai.com/blog/language-unsupervised/) — статья в блоге OpenAI про GPT-1.
7. [GPT-2](https://openai.com/blog/better-language-models/) — статья в блоге OpenAI про GPT-2.
8. [GPT-3](https://openai.com/blog/gpt-3-apps/) — статья в блоге OpenAI про GPT-3.
9. [WebGPT](https://openai.com/blog/improving-factual-accuracy/) — статья в блоге OpenAI про GPT-3, обученную гуглить.
10. [Codex](https://openai.com/blog/openai-codex/) — статья в блоге OpenAI про GPT-3, обученную писать код.

#NLP метрики

<font color='red'>Выбирайте метрику под свою конкретную задачу!</font> Или даже конструируйте её самостоятельно.

Помните эту картинку с предыдущей лекции? Сегодня мы разберём нейросетевые метрики.

**BLEURT, Prism** - нейросети, последние слои принимают на вход **эмбеддинги машинного и эталонного переводов**, а на выходе дают оценку качества перевода.

**COMET, UniTE** - нейросети, **эмбеддинги машинного и эталонного переводов**, **оригинал** переводимого текста.

**Безреференсные метрики** - модели, сравнивающие **напрямую машинный перевод и первоисточник** (reference-free metrics).

<img src ="https://habrastorage.org/getpro/habr/upload_files/2fa/8de/051/2fa8de0511cfaac2bf0fa12f8cc935eb.png" width="1000">

**BERTScore**

Одна из самых популярных метрик, [предложенная Zhang et al.](https://arxiv.org/pdf/1904.09675.pdf) в 2019 для оценки качества генерируемого текста. Основана на оценке близости контекстных эмбеддингов, полученных из предобученной нейросетевой модели BERT. Частично **решает проблему синонимов и опечаток** метрик BLEU и ROUGE.

Для расчета BERTScore близость двух предложений – сгенерированного моделью и эталонного – оценивается как сумма косинусных подобий между эмбеддингами слов, составляющих эти предложения.

<center><img src ="https://wiki.math.uwaterloo.ca/statwiki/images/6/6a/Architecture_BERTScore.PNG" width="1000"></center>

<center><em>Source: <a href="https://wiki.math.uwaterloo.ca/statwiki/index.php?title=BERTScore:_Evaluating_Text_Generation_with_BERT">The Illustrated GPT-2 (BERTScore: Evaluating Text Generation with BERT)</a></em></center>



На основе обоих токенов рассчитывается **Recall**, **Precision** и **F1**:

<center><img src ="https://wiki.math.uwaterloo.ca/statwiki/images/a/ad/Equations.PNG" width="1000"></center>

<center><em>Source: <a href="https://wiki.math.uwaterloo.ca/statwiki/index.php?title=BERTScore:_Evaluating_Text_Generation_with_BERT">The Illustrated GPT-2 (BERTScore: Evaluating Text Generation with BERT)</a></em></center>

Научимся это делать

In [ ]:
!pip install evaluate
from IPython.display import clear_output
clear_output()

Из важных параметров стоит [упомянуть](https://huggingface.co/spaces/evaluate-metric/bertscore) выбор языка и тип модели. По-умолчанию используется `roberta-large`, и, чтобы не загружать 1.4G Гб, мы выставляем более мелкую модель менее чем в 300 Мб.

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en", nthreads = -1, batch_size=128, model_type="distilbert-base-uncased")

In [ ]:
results

Отметим, что результат зависит от того, что мы принимаем за референс. Поэтому нам и выдают пару чисел.

[Перечень метрик и их объяснений](https://habr.com/ru/articles/745642/).

# Примеры применений Transformer


Непосредственное применение разобранной архитектуры Encoder-Decoder для перевода текста

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/transformer_text_translation_example.png" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em></center>

[The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)

## BERT (Bidirectional Encoder Representations from Transformers )

В случае BERT используется только Encoder часть

Это нейросеть, предобученная на огромном корпусе английского текста.

Перед ней ставили следующие задачи:

**Задача 1**

На вход дается предложение. В нем выбрано $15\%$ токенов, из которых:
1. $80\%$ замаскированы;
2. $10\%$ заменены случайным;
3. $10\%$ оставлены без изменений.

**Задача 2**

На вход даются два предложения. Необходимо определить, идет ли второе непосредственно за первым в тексте или нет (просто случайное предложение из корпуса).

В результате на вход подается все в таком виде:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/bert.jpg" width="800"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1810.04805.pdf">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></em></center>

$CLS$ токен нужен для того, чтобы нейросети было, куда класть информацию обо всем предложении в целом.

$SEP$ нужен просто для того, чтобы разделять два предложения (если мы подаем их два) и для того, чтобы отмечать окончание. Использование $SEP$ с двумя целями позволяет подавать в уже обученную нейросеть только одно предложение.




### Masked Language Model

Для первой задачи используем Encoder-Decoder

То есть у нас есть **Encoder**, который получает богатые представления, и добавленный только на время обучения **Decoder** (не attention, просто MLP).

Именно Decoder отвечает за то, чтобы предсказывать пропущенные/замененные токены. Ошибка считается только по тем $15\%$ токенов, для которых могли произойти изменения, а не по всему предложению.

Как гарантируется, что модель не заменяет имевшиеся в предложение слова на другие?

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/masked_language_model.png" width="800"></center>


### Next Sentence Prediction

Для второй задачи — Classifier.

При этом на вход ему подается только сам CLS токен


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/next_sentence_prediction.png" width="800"></center>


Обучаем модель на обеих задачах одновременно.


### Transfer learning с BERT

Обученную таким образом модель (оставляем только encoder), можно использовать для огромного числа других задач


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/transfer_learning_with_bert.jpg" width="800"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1810.04805.pdf">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></em></center>



### Zero-shot learning

Более того, есть BERT, тренированная на большом числе разных языков.

Это позволяет, например, сравнивать предложения из разных языков, хотя мы этому даже не учились. Поиграть можно [здесь](https://colab.research.google.com/github/deepmipt/dp_tutorials/blob/master/Tutorial_2_DeepPavlov_BERT_transfer_learning.ipynb#scrollTo=S1iqGcxUINyU). Веса будут грузиться ДОЛГО

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/zero_shot_learning_bert.png" width="800"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1810.04805.pdf">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></em></center>

### Специализированные аналоги BERT

Более того, можно тренировать BERT под строго определенные задачи — например, анализ текстов научных статей.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/bert_specialized_analogs.png" width="800"></center>

## GPT (Generative Pretrained Transformer )

В случае GPT используется только Decoder часть. Но теперь во всех частях используются masked attention.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/generative_pretrained_transformer_gpt.png" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em></center>

Это нейросеть обширно используется для задачи генерации текста.

Работает она следующим образом.

На вход подается "затравка" — какой-то текст (набор токенов). Можно подать просто SOS (Start of sentence) токен, обозначающий начало предложения и больше не несущий никакой дополнительной информации.

Сеть генерирует следующий токен. Добавляем его к входной последовательности и подаем этот удлиненный текст нейросети как вход.

Делаем так до тех пор, пока не надоест/не достигнем максимальной длины предложения/не встретим символ окончания генерации.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/gpt2_autoregression.gif" width="800"></center>

<center><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em></center>

Как ее обучали? На самом деле, тоже unsupervised learning. Но теперь перед моделью ставится задача предсказывать по предыдущим словам в предложении текущее.

Понятно, что такую модель можно сразу же идти и использовать для генерации, для которой она и училась.

Однако затем ее можно применять для многих других задач (опять же, путем transfer learning):

1. Классификации — подаем сразу все предложение, полученное представление используем для предсказания.

2. Entailment (Определение логического следования) — даем изначальные данные, гипотезу, надо оценить, следует ли гипотеза из данных.

3. Similarity — можем оценивать похожесть предложений. Так как это мы определяем порядок предложений, то, чтобы избежать неявной зависимости от порядка пропускания предложений, будем использовать результат пропускания через нейросеть обоих возможных порядков.

4. Выбор варианта ответа на вопросы — может обучить нейросеть отвечать на вопросы с множественным выбором.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/gpt_classification_entailment_similarity.png" width="1000">

Также можно научить нейросеть отвечать на вопросы и т.д.

Таким образом, мы можем даже текст переводить с помощью GPT.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_only_transformer_translation.png" width="800">

<em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em>

Просто подаем предложение с токеном в конце, определяющим, на какой язык переводим

По аналогичной схеме можем научить нашу сеть [делать summary текста](https://arxiv.org/abs/2109.10862)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_only_summarization.png" width="800">

<em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></em>

[Здесь](https://russiannlp.github.io/rugpt-demo/) можно поиграть с open-source русскоязычным аналогом GPT-3 от Сбера.

А [здесь](https://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb#scrollTo=e-NKauYvgTNG) — поиграть с GPT-J-6B, но уже в Collab

<font size = "6">Хорошие источники</font>

[Про трансформеры](https://www.notion.so/Transformers-969f4b27c48147778c1e2dbda0c83ce0)

[Аннотированный трансформер](http://nlp.seas.harvard.edu/2018/04/03/attention.html)

[Код множества моделей с красивыми комментариями](https://nn.labml.ai/)

[Зоопарк BERT](https://ai.plainenglish.io/so-how-is-bert-different-ad43a42cab48)

[Transformers in computer vision: ViT architectures, tips, tricks and improvements](https://theaisummer.com/transformers-computer-vision/)

[Illustrated transformer](https://jalammar.github.io/illustrated-transformer/)

[Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/)

[Open-source реализация GPT-3](https://arankomatsuzaki.wordpress.com/2021/06/04/gpt-j/)

[Transformer для русского языка](https://github.com/vlarine/transformers-ru)

[NLP Course for you](https://lena-voita.github.io/nlp_course.html)

[Курс по NLP от ШАД](https://github.com/yandexdataschool/nlp_course)

## Модели внимания в машинном переводе

Давайте посмотрим, как такой подход  работает на примере перевода с английского на французский.

На каждом шаге генерируется набор весов, которые отвечают за фокусировку на том или ином месте входной последовательности. Как мы видим, английское предложение имеет иной порядок слов относительно французского. Например, в английском варианте словосочетание **European Economic Area**, в то время как во французском **zone économique européenne**.

В английском прилагательные идут перед существительным, в то время как во французском языке наоборот.

Таким образом, благодаря гибкости модели мы можем обрабатывать и учитывать разный порядок слов в разных языках.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visualize_attention_weights.png" width="700">

<em>Source: <a href="https://arxiv.org/pdf/1409.0473.pdf">Neural machine translation by jointly learning to align and translate</a></em>



* Механизм внимания не обязательно должен принимать на вход последовательность.

* Мы можем применять его в том числе для генерации подписей для картинок. Входом в данном случае будет являться матрица признаков, которая была получена при применении сверточной сети к картинке.

* Далее по этой матрице мы считаем веса внимания и делаем аналогично первому примеру.

Следующим шагом будет также заметить, что для достаточно длинного предложения наша модель может забыть и то, что она генерирует. Потому мы можем сделать два attention: один — на представление исходного предложения, а второй — на представление того, что уже сгенерировано (что еще не сгенерировано заменяем нулями)


## Image Captioning with RNNs and Attention

Модели, основанные на внимании (attention), намного более продвинутые, нежели обычные нейросети. Они могут концентрироваться на отдельных частях данных, что позволяет избежать зашумления представлений.

Идея состоит в том, что на каждом этапе генерации описания нейронная сеть в разной степени обращает внимание на те или иные фрагменты изображения, соответствующие следующему слову в описании.

После обучения модели можно увидеть, что она как бы переносит своё внимание по изображению для каждого генерируемого слова.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_1.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_2.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_3.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_4.png" width="700">

<em>Source: <a href="http://cs231n.stanford.edu/slides/2022/lecture_11_ruohan.pdf">Stanford University CS231n: lectures</a></em>

**А если картинки?**

К примеру, у нас есть картинка. На этой картинки у нас есть области, которые можно описать одним словом — **key**. Например, фонарь/девушка/...

Сами эти области — это **value**, которые введенным **key** соответствуют.

Далее нам приходит **query**, например, running. Мы можем посчитать похожесть каждого из ключей, которые у нас есть, на query.

И далее выдать информацию только по **value**, похожим на наш **query**.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/key_query_value_example.jpg" width="800">


По сути мы описали то, как будем делать при помощи нейронных сетей питоновский словарь. С той разницей, что питоновский словарь может выдавать значения только для тех ключей, что в нем есть, а наш словарь выдает ответ для любого ключа-запроса, основываясь на его похожести на ключи словаря.

## Модели внимания в задаче генерации подписи к изображениям

Модель с вниманием также может быть применена в задаче, когда от нейронной сети требуется по изображению сгенерировать подпись.

Имеем набор пар "картинка : подпись"

Вместо рекуррентного кодировщика используем сверточную нейронную сеть. Веса внимания применяем к признакам на карте активации после нескольких сверточных слоев. Получается "маска" внимания.

Таким образом декодер имеет возможность обращать внимание на разные участки входного изображения при генерации очередного слова.

На иллюстрации приведены входные изображения и наложенные на них маски внимания, которые возникли в сети при генерации подчеркнутого слова.



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visulize_attention_map_examples.png" width="700">

<em>Source: <a href="https://arxiv.org/pdf/1502.03044.pdf">Show, Attend and Tell: Neural Image Caption Generation with Visual Attention</a></em>

Посмотрим, что “привлекает внимание” нейронной сети при написании текстового описания картинки.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visulize_attention_map.png" width="700">

<em>Source: <a href="https://arxiv.org/pdf/1502.03044.pdf">Show, Attend and Tell: Neural Image Caption Generation with Visual Attention</a></em>

Нейронные сети, использующие механизм внимания (attention), активно применяются для решения задачи [Visual Question Answering](https://paperswithcode.com/paper/vqa-visual-question-answering). В данной задаче нейросеть должна научиться давать развернутые ответы на вопросы по изображению. Модель должна не только решать задачу классификации, но и распознавать признаки (цвет, форма, размер, количество и т.д.) предметов на изображении, различать, в какой части изображения находится предмет и его положение относительно других предметов. Решение этой задачи может помочь людям с проблемами со зрением лучше ориентироваться в пространстве.

Подробнее:
1. [Нейросеть описывает мир незрячим людям](https://www.reg.ru/blog/nejroset-opisyvaet-mir-nezryachim-lyudyam/)
2. [Учим нейросети рассуждать о том, что они видят](https://www.reg.ru/blog/uchim-nejroseti-rassuzhdat-o-tom-chto-oni-vidyat/)